<a href="https://colab.research.google.com/github/santiagomvc/reconocimiento-patrones-unal/blob/master/Clasificacion_de_Imagenes_Resnet101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo Final Reconocimiento de Patrones - Clasificación de Imagenes

# **!!!Es recomendado correr este Notebook en Google Colab, de otra forma seran necesarios cambios para su correcta ejecución!!!**

## Objetivo

El objetivo principal de este desarrollo es aplicar los conceptos vistos en la materia Reconocimiento de Patrones  y en demas materias del posgrado en Analítica de la Universidad Nacional de Colombia Sede Medellín. Con este fin se desarrollo un sistema de clasificación de señales de transito con las siguientes características:

Objetivo General

*   Generar un Sistema de Clasificacion de Señales de transito basado en fotos reales recortadas de las señales.


Objetivos Específicos

*   Entrenar un algoritmo que permita de forma precisa (>90% Accuracy en development set) clasificar fotos de señales de transito (con distribuciones similares a la data de entrenamiento) en sus respectivas categorías. 
*   Crear una Inferfaz de usuario amigable para el uso interactivo del algoritmo que permita su entendimiento y validacion. 


Este tipo de sistemas, en conjunto con algoritmos adicionales de Computer Vision, pueden tener múltiples usos, entre los que se encuentran:


*   Evaluacion de señales por parte de vehículos autonomos para la toma de decisiones en la vía
*   Auditoría gubernamental de la cantidad y ubicación de las diferentes señales de transito en un área determinada  

## Metodología


Para el desarrollo del sistema de clasificación se decidió seguir la metodologia de proyectos de analítica CRISPDM, la cual consiste de 6 pasos

![Metodología CRISPDM](https://upload.wikimedia.org/wikipedia/commons/thumb/b/b9/CRISP-DM_Process_Diagram.png/800px-CRISP-DM_Process_Diagram.png)

## 1. Entendimiento del Problema

Con los avances técnicos de los últimos años en Vision por Computador (CV) y el aumento en la capacidad de almacenamiento de data no estructurada, han aumentado de forma significativa el número de aplicaciones industriales que requieren encontrar patrones en imagenes.

Dentro de estas aplicaciones se encuentra la clasificación de imagenes, que permite inferir la clase de un objeto con base en una imagen que lo represente.
La clasificacion de señales de transito, en conjunto con algoritmos de detección de objetos que permita identificar donde existen estas señales,  presenta múltiples aplicaciones incluyendo reconocimiento de señales para toma de decision de automóviles autonomos y auditoría de señales de transito por entes gubernamentales.

En este ejercicio nos concentraremos en la aplicación para la auditoría de señales, por lo que sera de ayuda una herramienta interactiva que permita comprender y validar el funcionamiento del modelo. 

![Traffic Sign Detection](https://www.itsinternational.com/_resources/assets/inline/custom/72/131413.jpg)

## 2. Entendimiento de la Data

Para la realización de este sistema y el entrenamiento del modelo se utilizó un dataset libre al público, el cual se puede descargar en el siguiente link: https://www.kaggle.com/c/ml-medellin-mar2019/data.

El dataset consiste de 27439 fotografías de señales de transito a color de 64x64 pixeles, cada una correspondiente a una categoría asociada al nombre del directorio que las almacena. En total existen 43 categorías posibles, que van desde señales de pare hasta límites de velocidad.

Es importante señalar que las imagenes utilizadas para el entrenamiento del modelo estan recortadas para incluir principalmente la información de las señales, por lo que para la aplicación en producción del sistema las imagenes de entrada deben ser preprocesadas manual o automáticamente para extraer las porciones que incluyan algún tipo de señal.

![Límite 20](https://i.ibb.co/jgfMdMG/27943933934972.jpg)

### Para correr en Colab recomendamos:
a. Cargar los datos preprocesados
1.   Descargar el archivo "ml-medellin-mar2019" aquí https://drive.google.com/file/d/1BPpBJn_ZwS3M5l-raaX1sd8X1JgJ8ouQ/view?usp=sharing
2.   Descomprimir el archivo descargado 
2.   Subir la carpeta generada a la raiz de Google Drive con el nombre "ml-medellin-mar2019"

b. Procesar los datos en Colab (Lento)
1.   Descargar la data de https://www.kaggle.com/c/ml-medellin-mar2019/data
2.   Descomprimir la carpeta descargada
3.   Descomprimir los archivos comprimidos dentro de la carpeta
4.   Subir la carpeta con los datos a la raiz de Google Drive con el nombre "ml-medellin-mar2019"
5.   Comentar las líneas no comentadas y descomentar las líneas comentadas en el bloque de codigo que comienza con "Reading images as vectors"

De no utilizar Colab recomendamos comentar las lineas relacionadas y cambiar los paths donde sea necesario.



In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Downloading not yet ready libraries
pip install scipy==1.1.0

In [3]:
# importing Libraries
import scipy
from PIL import Image
from scipy import ndimage
import os
import math
import matplotlib.pyplot as plt
from tensorflow.python.framework import ops
import pandas as pd
import tensorflow as tf
import numpy as np

from keras import layers, optimizers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Dropout
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
from sklearn.preprocessing import LabelBinarizer

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

Using TensorFlow backend.


In [0]:
# Defining paths
path = '/content/drive/My Drive/ml-medellin-mar2019'
train = path + '/train'

In [0]:
# Reading images as vectors 
#X = []
#Y = []
#for i in os.listdir(train):
#    label = i
#    print(label)
#    for j in  os.listdir(train + '/' + i):
#        fname = train + '/' + i + '/' + j
#        image = np.array(ndimage.imread(fname, flatten=False))
#        reshaped_image = scipy.misc.imresize(image, size=(64,64))
#        X.append(reshaped_image)
#        Y.append(int(label))


#X = np.array(X)
#Y = np.array(Y)

X = np.load(path + '/X.npy')
Y = np.load(path + '/Y.npy')

## 3. Data Preparation

Para el adecuado procesamiento de las imagenes y posterior entrenamiento del modelo es necesario almacenar las señales de transito de forma numérica, donde se utilizan tensores para representar los valores de los pixeles de cada imagen. Por tanto, una imagen a color de 64x64 pixeles es representada como un tensor de dimensiones (64,64, 3).
De forma mas general, un dataset con m ejemplos, h pixeles de altura, w pixeles de largo y c número de canales se representa con un tensor de la siguiente forma (m,h,w,c).
Por su parte, las categorías objetivo Y se representan con una matriz (num_clases, m) que permite comparación con las predicciones del algoritmo y el entrenamiento del modelo

Posterior a la representación  de los datos en forma de tensores, se normalizan los valores de los pixeles para facilitar el entrenamiento del modelo, el cual puede verse afectado de forma positiva por la representacion de los valores de entrada alrededor de 0. Para la normalizacion se dividen los valores de cada pixel por 255, que es el valor máximo que pueden presentar.

Por último, se procede a la separación del conjunto de datos m en dos conjuntos de datos diferentes, X_train,Y_train para el entrenamiento del modelo y X_test,Y_test para la validación de los resultados sobre un dataset no observado durante el entrenamiento.

Dado que la cantidad de datos es significativa, y que algunos algoritmos para modelamiento con imagenes necesitan una gran cantidad de datos, se decidio entrenar con el 95% de los datos y validar con el 5% restante. Es importante agregar que esta data es ordenada de forma aleatoria para que el modelo no aprenda relaciones espurias con base en la ubicación relativa de las imagenes.

![Images as Tensors](https://image.slidesharecdn.com/tensordecomposition-170301235239/95/a-brief-survey-of-tensors-5-638.jpg?cb=1488412458)



In [0]:
# Normalizando los valores
X = X/255

In [0]:
# Separando dev and test sets
dev_size = math.ceil(0.05 * X.shape[0])
dev_index = np.random.choice(list(range(0, X.shape[0])), dev_size, replace = False)
X_test = X[dev_index, :, :, :]
Y_test = Y[dev_index] 
X_train = np.delete(X, dev_index, axis = 0)
Y_train = np.delete(Y, dev_index, axis = 0)

shuffled_train = list(range(0, X_train.shape[0]))
np.random.shuffle(shuffled_train)
X_train = X_train[shuffled_train, :, :, :]
Y_train = Y_train[shuffled_train]

In [0]:
# Representando las clases
lb = LabelBinarizer()
Y_train = lb.fit_transform(Y_train)
Y_test = lb.fit_transform(Y_test)

## 4. Modelamiento

Para modelar las clases de las señales de transito con base en sus fotografías se decidio utilizar un modelo de redes neuronales (NN). Las redes neuronales son conjuntos de uno o mas nodos que representan una transformación lineal de un input A\[l] dados unos parametros (W\[l],b\[l]), seguida de una transformación no lineal del resultado g\[l] (g\[l] puede ser Tanh, ReLU,Sigmoid); donde los parametros (W\[l],b\[l]) son entrenados por medio del algoritmo de Backpropagation.

Debido a la gran cantidad de problemas abordados a través de NN existen múltiples arquitecturas que funcionan mejor dada la naturaleza del problema. Dado su éxito en tareas de modelamiento de imagenes se decidió utilizar redes neuronales convolucionales, las cuales son capaces de aprender filtros (W) capaces de extraer información de valor de las imagenes por medio de convoluciones. 

Especificamente se decidio utilizar un modelo basado en la arquitectura de Resnet101, la cual tiene más de 100 capas dentro de la red neuronal que se entrenan de forma adecuada gracias a su mecanismo de conexiones de atajo, que permite aprender funciones de identidad para múltiples capas y mejora los resultados de backpropagation. Además, esta arquitectura ha sido utilizada de forma exitosa en competencias como ImageNet y en datasets libres como CIFAR10.

Para la red neuronal se utilizaron los siguientes parametros:


* Arquitectura base: Resnet 101
* Número de capas: 101
* Learning Rate: 0.00005
* beta_1: 0.9
* beta_2: 0.999
* decay: 0.00005/25
* Optimizer: Adam
* Loss Function: categorical_crossentropy
* Epochs : 20
* Batch Size: 32


![Resnets](https://www.d2l.ai/_images/resnet-block.svg)

El siguiente codigo fue realizado con base en los ejercios de Deep Learning Specialization, Coursera.

In [0]:
def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    # Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X


In [0]:
def convolutional_block(X, f, filters, stage, block, s = 2):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path (≈3 lines)
    X = Conv2D(F2, (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(F3, (1, 1), strides = (1,1), name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv2D(F3, (1, 1), strides = (s,s), name = conv_name_base + '1', kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1' )(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X


In [0]:
def ResNet101(input_shape = (64, 64, 3), classes = 6):
    """
    Implementation of the popular ResNet101 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*22 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3 (≈4 lines)
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4 (≈23 lines)
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='g')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='h')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='i')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='j')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='k')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='l')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='m')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='n')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='o')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='p')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='q')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='r')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='s')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='t')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='u')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='v')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='w')


    # Stage 5 (≈3 lines)
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling2D((2,2), name = 'avg_pool')(X)

    # output layer
    X = Flatten()(X)
    #X = Dropout(rate = .8)(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet101')

    return model


In [12]:
model = ResNet101(input_shape = (64, 64, 3), classes = 43)

In [13]:
adam = optimizers.Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.00005/25, amsgrad=False)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
model.fit(X_train, Y_train, epochs = 20, batch_size = 32, shuffle = True)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/20
26067/26067 [==============================] - 142s 5ms/step - loss: 2.8493 - acc: 0.2333
Epoch 2/20
26067/26067 [==============================] - 122s 5ms/step - loss: 1.3829 - acc: 0.5726
Epoch 3/20
26067/26067 [==============================] - 122s 5ms/step - loss: 0.6578 - acc: 0.7913
Epoch 4/20
26067/26067 [==============================] - 122s 5ms/step - loss: 0.3568 - acc: 0.8865
Epoch 5/20
26067/26067 [==============================] - 121s 5ms/step - loss: 0.2315 - acc: 0.9245
Epoch 6/20
26067/26067 [==============================] - 122s 5ms/step - loss: 0.1794 - acc: 0.9425
Epoch 7/20
26067/26067 [==============================] - 121s 5ms/step - loss: 0.1284 - acc: 0.9587
Epoch 8/20
26067/26067 [==============================] - 120s 5ms/step - loss: 0.1087 - acc: 0.9652
Epoch 9/20
26067/26067 [==============================] - 121s 5ms/step - loss: 0.1051 - acc: 0.

## 5. Evaluación

Para la evaluación del modelo se utilizo la metrica de exactitud, que mide el total de las predicciones correctas sobre el total de las predicciones realizadas.

Al evaluar el modelo sobre la data de entrenamiento se encuentra que la exactitud es del 99.26%, mientras que en el dataset de desarrollo la exactitud es del 97.8%, con datos que el modelo no recibió durante el entrenamiento.

Por último, también se valido que no existieran saltos anormales en la función de perdida durante el entrenamiento del modelo

![Loss Functions](http://cs231n.github.io/assets/nn3/learningrates.jpeg)

In [15]:
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

1372/1372 [==============================] - 3s 2ms/step
Loss = 0.09141741659622585
Test Accuracy = 0.9759475218658892


In [0]:
# Writing Model
model.save(path + '/resnet_traffic_r101.h5')

## 6. Despliegue

Para el despliegue del sistema en pro de la validación de resultados y entendimiento del modelo por parte de usuarios finales se utilizo la libreria Dash de python, para realizar el despiegue de un tablero de control que permita cargar imagenes en línea de señales de transito e inferir la clase esperada.

Para explorar el tablero entrar a:
https://trafficdash.azurewebsites.net/

## Bibliografía


*   El modelo CRISP-DM: el nuevo plan para la minería de datos, almacenamiento de los datos. Shearer C., (2000); 5:13-22.
*   List Gradient-based learning applied to document recognition. Y LeCun, L Bottou, Y Bengio, P Haffner. Proceedings of the IEEE 86 (11), 2278-2324, 1998.
*   The Elements of Statistical Learning. T. Hastie, R. Tibshirani, and J. Friedman. Springer Series in Statistics Springer New York Inc., New York, NY, USA, (2001)
*   Deep Learning (Ian J. Goodfellow, Yoshua Bengio and Aaron Courville), MIT Press, 2016.
*   Deep Learning Specialization. Coursera.



